In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18150,2024-03-02,Taiwan Liga T1,05:30,Tainan TSG GhostHawks,Kaohsiung Aquas,3.25,1.31,205.5,1.85,1.85,0.0,0.00,0.00,n9zphXUs,0.307692,0.763359,0.540541,0.540541,0.071051,243.712,101.010976,0.414469,0.0,0.000000,NaN,177.11,2.622,1.258062,0.479810,-77.0,217.396,80.618700,0.370838,3.0,0.000000,0.000000,253.70,2.038,0.773253,0.379418,43.0,89,118,1.99,2.15,0.000,219.914,0.601661,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,2.60,0.520,0.596154,0.000000,0.0,0.000000
18151,2024-03-02,Taiwan Liga T1,03:00,New Taipei CTBC DEA,Taipei Taishin Mars,1.27,3.53,197.5,1.85,1.85,0.0,0.00,0.00,p0lIdDMQ,0.787402,0.283286,0.540541,0.540541,0.070688,135.492,26.123047,0.192801,1.2,1.643168,1.369306,124.95,1.406,0.190342,0.135378,3.0,290.252,91.866739,0.316507,1.2,1.643168,1.369306,167.04,3.290,1.380127,0.419491,-33.0,85,96,1.47,1.74,0.000,255.084,0.665859,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,-2.85,-0.570,-4.438596,0.000000,0.0,0.000000
18152,2024-03-02,Taiwan P. League+,06:00,Taoyuan Pauian Pilots,New Taipei Kings,1.97,1.76,186.5,1.85,1.81,1.5,1.83,1.62,KlNQ6RKE,0.507614,0.568182,0.540541,0.552486,0.075796,241.526,52.628100,0.217898,1.8,1.643168,0.912871,212.97,2.646,0.372934,0.140943,11.0,124.234,17.647073,0.142047,1.2,1.643168,1.369306,112.00,1.348,0.050695,0.037608,-12.0,93,80,2.29,1.40,196.468,142.740,0.079621,0.015456,0.086083,2.60,0.520,1.865385,0.0,0.0,0.0,-1.57,-0.314,-2.420382,0.000000,0.0,0.000000
18153,2024-03-02,Taiwan P. League+,03:30,Kaohsiung Steelers,Hsinchu Jko Lioneers,1.89,1.83,185.5,1.80,1.86,-1.5,2.05,1.96,8EJM7758,0.529101,0.546448,0.555556,0.537634,0.075549,295.708,77.514917,0.262133,1.2,1.643168,1.369306,316.80,3.118,0.797853,0.255886,-19.0,266.478,70.709060,0.265347,1.2,1.643168,1.369306,246.40,2.790,0.634902,0.227563,0.0,99,88,3.20,2.80,268.026,246.390,0.022810,0.023184,0.031740,0.19,0.038,23.421053,0.0,0.0,0.0,-1.56,-0.312,-2.660256,0.341498,0.4,0.058502
18154,2024-03-02,Suíça Liga Sb Feminina,13:30,Pully F,Alte Kanti Aarau F,1.44,2.65,139.5,1.84,1.86,0.0,0.00,0.00,fV7PISX6,0.694444,0.377358,0.543478,0.537634,0.071803,242.124,85.358105,0.352539,1.2,1.643168,1.369306,221.40,3.382,1.334717,0.394653,-14.0,267.466,327.851093,1.225767,0.6,1.341641,2.236068,106.86,4.258,5.200459,1.221338,-30.0,60,78,3.69,1.37,171.468,139.128,0.418386,0.007644,NaN,1.39,0.278,1.582734,0.0,0.0,0.0,-2.88,-0.576,-2.864583,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18544,2024-03-02,Europa Wbbl Feminina,06:45,RSU Merks F,Panevezys F,1.29,3.37,130.5,1.83,1.87,0.0,0.00,0.00,vPW1GCNT,0.775194,0.296736,0.546448,0.534759,0.071930,149.154,81.268883,0.544866,1.2,1.643168,1.369306,112.35,2.312,1.414486,0.611802,41.0,774.880,676.379443,0.872883,0.0,0.000000,NaN,1456.00,12.556,10.593495,0.843700,-122.0,107,56,1.05,26.00,0.000,0.000,0.631237,0.015289,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
18545,2024-03-02,Europa Liga Waba Feminina,13:00,Duga F,Cinkarna Celje F,9.95,22.00,150.5,1.85,1.85,0.0,0.00,0.00,KMbjMJzr,0.100503,0.045455,0.540541,0.540541,-0.854043,631.712,564.436302,0.893503,0.6,1.341641,2.236068

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
18181,22:00,Eua Nba,Memphis Grizzlies,Portland Trail Blazers,1.72,Back Home
18230,13:00,República Tcheca Zbl Feminina,Slavia Prague F,USK Prague F,1.64,Back Home
18239,12:30,Portugal Lfb Feminina,Natacao F,CAB Madeira F,1.79,Back Home
18243,14:00,Portugal Proliga,Santo André,Maia,1.79,Back Home
18249,22:00,Eua Ncaa,Idaho State,Northern Colorado,1.77,Back Home
18266,20:00,Eua Ncaa,Florida Intl,Western Kentucky,2.38,Back Home
18268,20:00,Eua Ncaa,Boston College,Pittsburgh,1.95,Back Home
18274,11:00,Suécia Superettan,Huddinge,Sloga Uppsala,1.66,Back Home
18301,18:00,Eua Ncaa,Albany,Maine,1.80,Back Home
18321,18:00,Eua Ncaa,Charlotte,South Florida,1.61,Back Home
